## Testing Model

In [2]:
import numpy as np # untuk operasi numerik dan komputasi array multidimensi secara efisien.
import pandas as pd # untuk manipulasi dan analisis data 
import pickle # # untuk serialisasi dan deserialisasi objek Python.
import tensorflow as tf # untuk machine learning dan neural network.

In [30]:
# Load and preprocess data
loadedModel = pd.read_hdf('wisata_model_baru.h5', 'df')
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
data_model = pd.read_csv('destinasi_wisata_baru.csv')
data_model

,Unnamed: 0,Place_Id,Place_Name,Description,Category,City,Price,Rating
0,0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6
1,1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6
2,2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6
3,3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5
4,4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5
...,...,...,...,...,...,...,...,...
432,432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000,4.4
433,433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0,4.6
434,434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0,4.4
435,435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0,4.6


In [31]:
# Select relevant features for dataframe
data_model = loadedModel[['Place_Name','City', 'Price','Rating', 'Category']]  # create a copy to avoid SettingWithCopyWarning

In [32]:
# Handle duplicates and missing data
data_model.drop_duplicates(inplace=True)
data_model.dropna(inplace=True)

C:\Users\dinar\AppData\Local\Temp\ipykernel_22580\1167761378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model.drop_duplicates(inplace=True)
C:\Users\dinar\AppData\Local\Temp\ipykernel_22580\1167761378.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model.dropna(inplace=True)


In [39]:
# Function for recommendation based on city
def recommend_places(city, top_n=5): #top_n = wisata
    # Filter the data based on the input city 
    recommendations = data_model[(data_model['City'].str.contains(city, case=False, na=False))]
    recommendations.sort_values(by='Rating', ascending=True, inplace=True)

    # Return the top_n results
    return recommendations.head(top_n)

In [41]:
# sample input
city = 'Surabaya'

recommendations = recommend_places(city)
print(recommendations)

                                Place_Name      City  Price  Rating  \
391            Ekowisata Mangrove Wonorejo  Surabaya      0       2   
416                 Klenteng Sanggar Agung  Surabaya  10000       2   
417  Museum Sepuluh Nopember Kota Surabaya  Surabaya   5000       2   
418                  Monumen Tugu Pahlawan  Surabaya      0       2   
419         Surabaya Museum (Gedung Siola)  Surabaya      0       2   

       Category  
391  Cagar Alam  
416      Budaya  
417      Budaya  
418      Budaya  
419      Budaya  


C:\Users\dinar\AppData\Local\Temp\ipykernel_22580\2078868230.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations.sort_values(by='Rating', ascending=True, inplace=True)


In [42]:
# Function for recommendation based on city and budget
def recommend_places_2(city, budget, top_n=5): #top_n = wisata
    # Filter the data based on the input city and budget
    harga = data_model['Price'] * top_n
    recommendations = data_model[(data_model['City'].str.contains(city, case=False, na=False)) & (harga <= budget)]

    # Calculate the absolute difference between the budget and the price, then sort the data based on this difference
    recommendations['abs_diff'] = abs(recommendations['Price'] - budget)
    recommendations.sort_values(by='abs_diff', ascending=True, inplace=True)

    # Return the top_n results
    return recommendations.head(top_n)

# Usage:
city = 'Jakarta'
budget = 150000

recommendations = recommend_places_2(city, budget)
print(recommendations)

                    Place_Name     City  Price  Rating       Category  \
61    Taman Legenda Keong Emas  Jakarta  30000       2  Taman Hiburan   
30  Wisata Alam Mangrove Angke  Jakarta  25000       2     Cagar Alam   
55                Pantai Ancol  Jakarta  25000       2         Bahari   
5      Taman Impian Jaya Ancol  Jakarta  25000       2  Taman Hiburan   
66      Margasatwa Muara Angke  Jakarta  25000       2     Cagar Alam   

    abs_diff  
61    120000  
30    125000  
55    125000  
5     125000  
66    125000  


C:\Users\dinar\AppData\Local\Temp\ipykernel_22580\3665265690.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['abs_diff'] = abs(recommendations['Price'] - budget)
C:\Users\dinar\AppData\Local\Temp\ipykernel_22580\3665265690.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations.sort_values(by='abs_diff', ascending=True, inplace=True)
